In [ ]:
readme = open("aclImdb/README").read()
print(readme)

Large Movie Review Dataset v1.0

Overview

This dataset contains movie reviews along with their associated binary
sentiment polarity labels. It is intended to serve as a benchmark for
sentiment classification. This document outlines how the dataset was
gathered, and how to use the files provided. 

Dataset 

The core dataset contains 50,000 reviews split evenly into 25k train
and 25k test sets. The overall distribution of labels is balanced (25k
pos and 25k neg). We also include an additional 50,000 unlabeled
documents for unsupervised learning. 

In the entire collection, no more than 30 reviews are allowed for any
given movie because reviews for the same movie tend to have correlated
ratings. Further, the train and test sets contain a disjoint set of
movies, so no significant performance is obtained by memorizing
movie-unique terms and their associated with observed labels.  In the
labeled train/test sets, a negative review has a score <= 4 out of 10,
and a positive review has a scor

In [ ]:
!wget "http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz"

--2020-12-19 16:54:00--  http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
Resolving ai.stanford.edu (ai.stanford.edu)... 171.64.68.10
Connecting to ai.stanford.edu (ai.stanford.edu)|171.64.68.10|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 84125825 (80M) [application/x-gzip]
Saving to: ‘aclImdb_v1.tar.gz’

aclImdb_v1.tar.gz   100%[===================>]  80.23M  18.2MB/s    in 11s     

2020-12-19 16:54:11 (7.47 MB/s) - ‘aclImdb_v1.tar.gz’ saved [84125825/84125825]



In [ ]:
%ls

aclImdb_v1.tar.gz  sample_data/  sentidb  sentidb.tar.gz


In [ ]:
!sudo tar -xvf aclImdb_v1.tar.gz

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import GaussianNB

In [ ]:
import os

In [ ]:
pospath = "aclImdb/train/pos"
train_pos_files = os.listdir(pospath)
print(train_pos_files[:10])

['1532_10.txt', '742_9.txt', '11346_10.txt', '5597_10.txt', '6637_8.txt', '7922_7.txt', '9943_7.txt', '9822_10.txt', '7928_10.txt', '1612_10.txt']


In [ ]:
  mytemp = open(pospath+"/"+train_pos_files[0]).read()
  print(mytemp)
  temprating = int((train_pos_files[0].split('_')[1]).split('.')[0])
  print(temprating)

This, like Murder She Wrote, is one of those shows, that after a stressful day at school, I sit down in front of the TV, and watch. Why? Because I genuinely enjoy it, and it's a shame it's not on the air anymore. Dick Van Dyke is amazing as Dr Mark Sloan, a doctor-turned-detective, who with his son, solves murders. He is joined by a largely unknown but very competent supporting cast, namely his real-life son, Barry Van Dyke. Victoria Rowell is also good, but I noticed that every series her hairstyle changes. i also liked Scott Baio and Charlie Schlatter, but I particularly loved Michael Tucci as Norman, and was puzzled how he suddenly disappeared. This show is so entertaining,with great guest stars, it's a bit obvious at times, like Colombo, but in every episode, there is always something to chuckle about. In conclusion, a great series, with two thumbs up and a 10/10. Bethany Cox
10


In [ ]:
myreview = []
myrating = []
for filename in train_pos_files:
  mytemp = open(pospath+"/"+filename)
  tempreview = mytemp.read()
  myreview.append(tempreview)
  mytemp.close()
  temprating = int((filename.split('_')[1]).split('.')[0])
  myrating.append(1)


print(myreview[0])
print(myrating[0])

This, like Murder She Wrote, is one of those shows, that after a stressful day at school, I sit down in front of the TV, and watch. Why? Because I genuinely enjoy it, and it's a shame it's not on the air anymore. Dick Van Dyke is amazing as Dr Mark Sloan, a doctor-turned-detective, who with his son, solves murders. He is joined by a largely unknown but very competent supporting cast, namely his real-life son, Barry Van Dyke. Victoria Rowell is also good, but I noticed that every series her hairstyle changes. i also liked Scott Baio and Charlie Schlatter, but I particularly loved Michael Tucci as Norman, and was puzzled how he suddenly disappeared. This show is so entertaining,with great guest stars, it's a bit obvious at times, like Colombo, but in every episode, there is always something to chuckle about. In conclusion, a great series, with two thumbs up and a 10/10. Bethany Cox
1


In [ ]:
negpath = "aclImdb/train/neg"
train_neg_files = os.listdir(negpath)

for filename in train_neg_files:
  mytemp = open(negpath+"/"+filename)
  tempreview = mytemp.read()
  myreview.append(tempreview)
  mytemp.close()
  temprating = int((filename.split('_')[1]).split('.')[0])
  myrating.append(-1)


print(myreview[-1])
print(myrating[-1])

This is the worst thing the TMNT franchise has ever spawned. I was a kid when this came out and I still thought it was deuce, even though I liked the original cartoon.<br /><br />There's this one scene I remember when the mafia ape guy explains to his minions what rhetorical questions are. It's atrocious. Many fans hate on the series for including a female turtle, but that didn't bother me. So much so that I didn't even remember her until I read about the show recently. All in all, it's miserably forgettable.<br /><br />The only okay thing was the theme song. Guilty pleasure, they call it... Nananana ninja...
-1


In [ ]:
mydbreview = pd.Series(myreview)
mydbreview.head()

0    This, like Murder She Wrote, is one of those s...
1    Coming of age movies are quite usual these day...
2    I grew up with H.R. Pufnstuff and the dashingl...
3    "Le Locataire"("The Tenant")is without a doubt...
4    Story says that on that on December 28, 1895, ...
dtype: object

In [ ]:
mydbrating = pd.Series(myrating)
mydbrating.head()

0    1
1    1
2    1
3    1
4    1
dtype: int64

In [ ]:
# bring rating between -5 to +5
mydbratingnormalized = mydbrating.sub(5)
mydbratingnormalized.head()

0    5
1    4
2    5
3    5
4    3
dtype: int64

In [ ]:
# get test data in required form

myreviewtest = []
myratingtest = []

testpospath = "aclImdb/test/pos"
testnegpath = "aclImdb/test/neg"

test_pos_files = os.listdir(testpospath)
test_neg_files = os.listdir(testnegpath)

for filename in test_pos_files:
  mytemp = open(testpospath+"/"+filename)
  tempreview = mytemp.read()
  myreviewtest.append(tempreview)
  mytemp.close()
  temprating = int((filename.split("_")[1]).split(".")[0]) - 5
  myratingtest.append(1)

print(myreviewtest[-1])
print(myratingtest[-1])

for filename in test_neg_files:
  mytemp = open(testnegpath+"/"+filename)
  tempreview = mytemp.read()
  myreviewtest.append(tempreview)
  mytemp.close()
  temprating = int((filename.split("_")[1]).split(".")[0]) - 5
  myratingtest.append(-1)

print(myreviewtest[-1])
print(myratingtest[-1])

"The Aristocats" is classic Disney at it's best. It's not considered as the ultimate Disney classic along side the more well-known Disney-films, but it's a well-made and fun film that certainly deserves to be a Disney classic. "Aristocats" is certainly a sure kids-pleaser, cause it has all the components for a fun kids film. I'll liked it very much as a kid. I'm not sure if this film is a sure pleaser to the adults, who could watch Disney classics for the nostalgia sake, but I still enjoy this film as an young adult.<br /><br />Storywise, "Aristocats" may not be so complex or innovative, but it doesn't matter. This film was made on that period where the filmmakers followed the motto that the characters were the story and wise versa, which "Aristocats" certainly shows. The characters make the film; they're all likable, fun, have great lines and have a great interaction with each other. The voice actors are also a part for making the characters so compelling (Eva Gabor as Duchess, Phil H

In [ ]:
mydbreviewtest = pd.Series(myreviewtest)
mydbratingtest = pd.Series(myratingtest)

In [ ]:
tfidf_vectorizer = TfidfVectorizer(stop_words='english', max_df=0.9)
tfidf_train = tfidf_vectorizer.fit_transform(mydbreview)
tfidf_test = tfidf_vectorizer.transform(mydbreviewtest)

In [ ]:
from sklearn.naive_bayes import MultinomialNB
mynbc = MultinomialNB()
mynbc.fit(tfidf_train, mydbrating)
rating_pred = mynbc.predict(tfidf_test)

In [ ]:
accuracy_score(mydbratingtest, rating_pred)

0.82992

In [ ]:
confusion_matrix(mydbratingtest, rating_pred)

array([[10979,  1521],
       [ 2731,  9769]])